In [100]:
import pandas as pd
import zipfile
import json
import plotly.express as px
import numpy as np
from scipy.signal import filtfilt
from scipy.signal import find_peaks
import plotly.graph_objs as go

In [14]:
zip_file_path = 'squats_jumps/20221025T093638_U001_SQUAT_JUMP.zip'
json_file_name = '20221025T093638_U001_SQUAT_JUMP.json'

with zipfile.ZipFile(zip_file_path, 'r') as z:
    with z.open(json_file_name) as f:
        json_data = json.loads(f.read())

In [22]:
df = pd.DataFrame(json_data["data"])
window_size = 5

b = np.ones(window_size) / window_size
df['linAccZ_Smooth'] = filtfilt(b, 1, df['linAccZ'])
df

,time,accX,accY,accZ,gyroX,gyroY,gyroZ,magnX,magnY,magnZ,linAccX,linAccY,linAccZ,exercise,linAccZ_Smooth
0,1970-01-01 00:00:00.000,-0.337400,9.384985,2.451261,-16.030001,6.30,0.35,-58.336884,151.008179,69.075043,-0.045889,0.334177,-0.107133,NO_EXERCISE,-0.107133
1,1970-01-01 00:00:00.009,-0.435509,9.392164,2.520655,-13.020000,6.65,0.77,-57.398693,150.610764,71.241280,0.055924,0.308101,-0.077707,NO_EXERCISE,-0.092686
2,1970-01-01 00:00:00.018,-0.476189,9.528560,2.381866,-12.740000,6.86,0.98,-57.710804,152.783325,71.207939,-0.034385,0.139919,0.025234,NO_EXERCISE,-0.082098
3,1970-01-01 00:00:00.027,-0.413973,9.495059,2.298115,-14.000000,6.86,0.84,-58.869091,153.492996,69.799446,-0.140626,0.101335,-0.030596,NO_EXERCISE,-0.084841
4,1970-01-01 00:00:00.036,-0.433116,9.382592,2.393831,-14.000000,7.21,0.63,-56.672897,149.408096,69.761551,-0.062455,0.162743,-0.115754,NO_EXERCISE,-0.098436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,1970-01-01 00:00:48.953,1.117488,9.499845,1.549134,10.290001,3.99,1.89,-51.414738,156.545090,76.458725,0.177307,0.309688,-0.123782,NO_EXERCISE,-0.162529
4988,1970-01-01 00:00:48.962,1.040915,9.629062,1.388809,9.170000,2.87,1.75,-50.048840,152.600540,76.311089,0.170717,0.124593,-0.025303,NO_EXERCISE,-0.091525
4989,1970-01-01 00:00:48.971,0.978699,9.758279,1.240449,6.650000,1.40,2.10,-47.188889,150.298737,79.528725,0.157191,-0.045870,0.076714,NO_EXERCISE,-0.019415
4990,1970-01-01 00:00:48.980,1.021772,9.794172,1.211734,3.780000,1.05,2.52,-47.458611,152.480240,80.758354,0.112654,-0.050407,0.113528,NO_EXERCISE,0.036874


In [71]:
peaks, _ = find_peaks(df['linAccZ_Smooth'])

In [87]:
def extract_exercises(series, peaks_list: list, threshold=2):
    exercises = []
    current_exercise = []
    
    index = 0
    is_add = False
    
    while index < len(series):
        info = series[index]
        if is_add:
            if index in peaks_list and abs(info['linAccZ_Smooth']) < threshold:
                exercises.append(current_exercise)
                current_exercise = []
                is_add = False
            else:
                current_exercise.append(info)
        else:
            if len(peaks_list) >= 2 and index in peaks_list:
                if abs(info['linAccZ_Smooth']) < threshold < abs(series[peaks_list[1]]['linAccZ_Smooth']):
                    is_add = True
        if index in peaks_list:
            peaks_list.remove(index)
        index += 1
    return exercises

In [88]:
squat_jumps = extract_exercises(df.to_dict(orient='records'), peaks.tolist())

In [103]:
squat_jump = pd.DataFrame(squat_jumps[0])
squat_jump = squat_jump.set_index('time')
squat_jump = squat_jump.drop("exercise", axis=1)
squat_jump

,accX,accY,accZ,gyroX,gyroY,gyroZ,magnX,magnY,magnZ,linAccX,linAccY,linAccZ,linAccZ_Smooth
time,,,,,,,,,,,,,
1970-01-01 00:00:01.990,-0.114860,9.576418,2.511083,10.780001,0.000000,3.08,-58.821533,154.263565,68.533867,-0.257942,0.146811,0.089440,-0.063147
1970-01-01 00:00:01.999,0.143574,9.624276,2.156933,5.110000,-0.980000,3.71,-57.345104,152.702759,69.789345,-0.645624,-0.025382,0.035797,-0.162343
1970-01-01 00:00:02.008,0.239291,9.547703,1.821926,0.770000,-2.520000,3.85,-57.048767,153.446945,69.779091,-0.865807,-0.244508,-0.126033,-0.311445
1970-01-01 00:00:02.017,0.138789,9.260554,1.563492,3.080000,-3.290000,3.43,-55.704670,151.097717,69.774208,-0.845361,-0.452055,-0.463846,-0.494654
1970-01-01 00:00:02.026,-0.019143,8.846581,1.297879,16.029999,-4.060000,2.87,-55.476364,153.987061,70.845604,-0.750949,-0.651624,-0.921804,-0.684997
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 00:00:03.755,-0.631728,8.356034,1.527598,-18.200001,-15.470001,-0.63,-57.049683,151.925583,75.116852,0.729880,0.210635,-1.323475,-1.563609
1970-01-01 00:00:03.770,-0.404401,9.741529,1.226092,-20.650000,-15.540001,-1.19,-53.807697,151.907150,75.512421,0.281568,-0.115312,0.014658,-0.579620
1970-01-01 00:00:03.779,-0.512082,10.842265,0.647008,-27.160000,-12.180000,-2.45,-52.187412,148.977722,75.262650,0.001508,-0.824080,1.034956,0.213396


In [104]:
fig = px.line(squat_jump, y='linAccZ_Smooth', title='Linear Acceleration Z over Time')
fig.show()

In [111]:
peaks, _ = find_peaks(squat_jump['linAccZ_Smooth'])

fig = go.Figure()
fig.add_trace(go.Scatter(x=squat_jump.index, y=squat_jump['linAccZ_Smooth'], mode='lines', name='Time Series'))
fig.add_trace(go.Scatter(x=squat_jump.index[peaks], y=squat_jump['linAccZ_Smooth'][peaks], mode='markers', marker=dict(size=10, color='red'), name='Peaks'))
fig.update_layout(title='Time Series Data with Peaks', xaxis_title='Time', yaxis_title='Value')
fig.show()

C:\Users\josec\AppData\Local\Temp\ipykernel_6900\3929756767.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

